# Topic Modeling
In this exercise, we will do topic modeling with gensim. Use the [topics and transformations tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html) as a reference.

In [1]:
import os
from collections import defaultdict

import gensim
import nltk

For tokenizing words and stopword removal, download the NLTK punkt tokenizer and stopwords list.

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\YaFr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YaFr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

First, we load the [Lee Background Corpus](https://hekyll.services.adelaide.edu.au/dspace/bitstream/2440/28910/1/hdl_28910.pdf) included with gensim that contains 300 news articles of the Australian Broadcasting Corporation.

In [3]:
from gensim.test.utils import datapath
train_file = datapath('lee_background.cor')
articles_orig = open(train_file).read().splitlines()

Preprocess the text by lowercasing, removing stopwords, stemming, and removing rare words.

In [7]:
# define stopword list
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords = stopwords | {'\"', '\'', '\'\'', '`', '``', '\'s'}

# initialize stemmer
stemmer = nltk.stem.PorterStemmer()

def preprocess(article):
    # tokenize
    article = nltk.word_tokenize(article)  # TODO

    # lowercase all words
    article = [word.lower() for word in article]

    # remove stopwords
    article = [word for word in article if word not in stopwords]  # TODO

    # optional: stem
    # article = [stemmer.stem(word) for word in article]
    return article

articles = [preprocess(article) for article in articles_orig]

# create the dictionary and corpus objects that gensim uses for topic modeling
dictionary = gensim.corpora.Dictionary(articles)

# remove words that occur in less than 2 documents, or more than 50% of documents
dictionary.filter_extremes(no_below=2, no_above=0.5)
temp = dictionary[0]  # load the dictionary by calling it once
corpus_bow = [dictionary.doc2bow(article) for article in articles]


Now we create a TF-IDF model and transform the corpus into TF-IDF vectors.

In [8]:
model_tfidf = gensim.models.TfidfModel(corpus_bow) # step 1 -- initialize a model
corpus_tfidf = model_tfidf[corpus_bow]
print(corpus_tfidf[0])
print(corpus_bow[0])

[(0, 0.045163832296308125), (1, 0.049004990699027966), (2, 0.09398031720792203), (3, 0.06797874731615453), (4, 0.08637534553463992), (5, 0.10158528888120417), (6, 0.058872481173046734), (7, 0.045871696227162966), (8, 0.04660732651093343), (9, 0.03476708703034139), (10, 0.09174339245432593), (11, 0.06379342938648586), (12, 0.08097953226203827), (13, 0.08637534553463992), (14, 0.06576958891547403), (15, 0.05748249959948285), (16, 0.07679421433236962), (17, 0.09398031720792203), (18, 0.04197717742438698), (19, 0.06379342938648586), (20, 0.09398031720792203), (21, 0.07679421433236962), (22, 0.08097953226203827), (23, 0.058872481173046734), (24, 0.05497796237027076), (25, 0.05497796237027076), (26, 0.07337456058875615), (27, 0.05497796237027076), (28, 0.08637534553463992), (29, 0.058872481173046734), (30, 0.062005775644911734), (31, 0.08637534553463992), (32, 0.09398031720792203), (33, 0.04737299069698862), (34, 0.07048328454536662), (35, 0.09398031720792203), (36, 0.09398031720792203), (37

Now we train an [LDA model](https://radimrehurek.com/gensim/auto_examples/tutorials/run_lda.html) with 10 topics on the TF-IDF corpus. Save it to a variable `model_lda`.

In [9]:
model_lda = gensim.models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=10)

Let's inspect the first 5 topics of our model.

In [10]:
model_lda.print_topics(5)

[(4,
  '0.002*"company" + 0.002*"timor" + 0.002*"zimbabwe" + 0.002*"south" + 0.002*"fire" + 0.002*"union" + 0.001*"mr" + 0.001*"action" + 0.001*"commonwealth" + 0.001*"wales"'),
 (8,
  '0.002*"indian" + 0.002*"detainees" + 0.002*"man" + 0.001*"government" + 0.001*"arafat" + 0.001*"india" + 0.001*"minister" + 0.001*"asked" + 0.001*"car" + 0.001*"qantas"'),
 (9,
  '0.002*"palestinian" + 0.002*"israeli" + 0.002*"fire" + 0.001*"australian" + 0.001*"government" + 0.001*"australia" + 0.001*"india" + 0.001*"force" + 0.001*"income" + 0.001*"foreign"'),
 (6,
  '0.002*"palestinian" + 0.002*"economy" + 0.001*"australian" + 0.001*"economic" + 0.001*"hewitt" + 0.001*"reid" + 0.001*"israeli" + 0.001*"mr" + 0.001*"lee" + 0.001*"know"'),
 (3,
  '0.002*"australian" + 0.002*"afghanistan" + 0.002*"hicks" + 0.002*"palestinian" + 0.001*"test" + 0.001*"guilty" + 0.001*"rates" + 0.001*"mr" + 0.001*"confirmed" + 0.001*"world"')]

We see the 5 topics with the highest importance. For each topic, the 10 most important words are shown, together with their coefficient of "alignment" to the topic.

## Document Similarity
We now use our LDA model to compare the similarity of new documents (*queries*) to documents in our collection.

First, create an index of the news articles in our corpus. Use the `MatrixSimilarity` transformation as described in gensim's [similarity queries tutorial](https://radimrehurek.com/gensim/auto_examples/core/run_similarity_queries.html).

In [11]:
index = gensim.similarities.MatrixSimilarity(model_lda[corpus_tfidf])

Now, write a function that takes a query string as input and returns the LDA representation for it. Make sure to apply the same preprocessing as we did to the documents.

In [12]:
def get_lda_vector(text):
    text = preprocess(text)
    bow = dictionary.doc2bow(text)
    tfidf = model_tfidf[bow]
    lda_vector = model_lda[tfidf]
    return lda_vector

lda_vector = get_lda_vector("A new bill sparked massive protests in Israel, as it would massively limit the powers of the judiciary.")

Print the top 5 most similar documents, together with their similarities, using your index created above.

In [13]:
document_similarities = index[lda_vector]
for doc_idx, doc_similarity in sorted(enumerate(document_similarities), key=lambda x: x[1], reverse=True)[:5]:
    print(doc_similarity, articles_orig[doc_idx])

0.9803847 Israel launched massive air raids across the West Bank and Gaza Tuesday, piling pressure on Yasser Arafat with a rocket strike on  a police post next to his offices, after Prime Minister Ariel Sharon branded his administration a "sponsor of terrorism". Israeli F-16 warplanes launched a series of strikes on Gaza City, while Apache helicopters fired rockets on Palestinian security offices in Khan Yunis in the southern Gaza Strip and on the West Bank towns of Salfit and Tulkarem. They also fired missiles on a security post just metres from Mr Arafat's offices in Ramallah, but the Palestinian leader, who was in his office at the time, was unhurt. But two policemen were slightly wounded, officials said. Israeli army spokesman Brigadier General Ron Kitrey said Mr Arafat was not targeted. Two people were killed in the Gaza strikes and around 120 injured, half of them schoolboys, Palestinian hospital officials said. The attacks came as Israel's Foreign Minister Shimon Peres said he d

Run your code again, now training an LDA model with 100 topics. Do you see a qualitative difference in the top-5 most similar documents?